## Jupyter configurations

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Imports

In [49]:
import tensorflow as tf
import keras
import keras.backend as K
import numpy as np
import pandas as pd
import preprocessing as pp
import sys, inspect, argparse, importlib, traceback

importlib.reload(pp)

<module 'preprocessing' from 'H:\\gsq-metadata-extraction\\filepath-metadata-extraction\\preprocessing.py'>

## Accuracy Metric

In [5]:
# percentage of samples that exactly match
def exact_match_accuracy(y_true, y_pred):
    argmax_true = tf.math.argmax(y_true, axis=2)            # onehot to index               (batch, width, onehot:int) -> (batch, width:int)
    argmax_pred = tf.math.argmax(y_pred, axis=2)            # onehot to index               (batch, width, onehot:int) -> (batch, width:int)
    match_char = tf.math.equal(argmax_true, argmax_pred)    # match characters              (batch, width:int) -> (batch, width:bool)
    match_word = tf.math.reduce_all(match_char, axis=1)     # require all character in sample to match      (batch, width:bool) -> (batch:bool)
    match_int = tf.cast(match_word, tf.float32)             # bool to int                                   (batch:bool) -> (batch:int)
    return tf.reduce_mean(match_int)                        # percentage of samples that are an exact match (batch:int) -> int

## Log function

In [6]:
verbose = False
def log(*l, **d): 
    if verbose: print(*l, **d)
        
training_history = []

def training_log(x, y, a, b, e, l, m):
    training_history.append({'x':x, 'y':y, 'architecture':a, 'batch size':b, 'epochs':e, 'loss':l, 'accuracy':m})

## Load Data

In [ ]:
data = pp.load('training_data.p')
voc_size = pp.char_count

## Preprocess Data

### Tokens and Characters

In [7]:
# tokens used to communicate non character entities
# tokens = ['<Padding>', '<Go>', '<EndOfString>', '<UnknownChar>', '<SurveyNum>', '<SurveyName>', '<LineName>', '<SurveyType>', '<PrimaryDataType>', '<SecondaryDataType>', '<TertiaryDataType>', '<Quaternary>', '<File_Range>', '<First_SP_CDP>', '<Last_SP_CDP>', '<CompletionYear>', '<TenureType>', '<Operator Name>', '<GSQBarcode>', '<EnergySource>', '<LookupDOSFilePath>', '<Source Of Data>']
tokens = ['<Padding>', '<Go>', '<EndOfString>', '<UnknownChar>']

# get set of characters to be used, use static preset list of characters
#available_chars = list(" ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890-_().,\\/\"':&")
available_chars = list(" ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890-_().,\\/\"':&")

# generate character to int and int to character maps
char_to_int = {c: i for i, c in enumerate(tokens + available_chars)}
int_to_char = {i: c for c, i in char_to_int.items()}
char_count = len(char_to_int) # number of character available

### Read raw data

In [131]:
raw_source_file = 'SHUP 2D Files Training Data.csv'

# read raw training data
data_df = pd.read_csv(raw_source_file, dtype=str)
data = {feature:data_df[feature].values for feature in data_df.columns.values}

data['LookupDOSFilePath_Words'] = np.array([s.split('\\')[2:] for s in data['LookupDOSFilePath']])
data['FileName_Words'] = np.array([s.split('_') for s in data['FileName']])

print(data.keys())

dict_keys(['Unique Record ID', 'FileName', 'Original_FileName', 'SurveyNum', 'SurveyName', 'LineName', 'SurveyType', 'PrimaryDataType', 'SecondaryDataType', 'TertiaryDataType', 'Quaternary', 'File_Range', 'First_SP_CDP', 'Last_SP_CDP', 'CompletionYear', 'TenureType', 'Operator Name', 'GSQBarcode', 'EnergySource', 'LookupDOSFilePath', 'Source Of Data', 'LookupDOSFilePath_Words', 'FileName_Words'])


### Vectorize Strings

In [57]:
def vectorize_data(data):
    if type(data) == str:
        return [char_to_int[char] for char in data.upper()]
    
    else:
        try:
            return [vectorize_data(d) for d in data]
        except Exception as e:
            #traceback.print_exc()
            #print(data, type(data))
            return []

def decode_vector(vector):
    return ''.join([int_to_char[int(i)] for i in vector])

def decode_data(data):
    return [decode(v) for v in data]



### Add Padding Tokens

In [75]:
# find size of largest array across each dimension to computer shape of bounding ndarray
def size(data):
    
    if type(data) == int:
        return ()
    
    this_size = len(data)
    
    if this_size > 0:
        inner_sizes = np.array([size(d) for d in data])
        inner_sizes = tuple(np.amax(inner_sizes, axis=0))
    else:
        inner_sizes = ()
    
    return (this_size,) + inner_sizes
    
    
def insert(matrix, data, indices):
    if type(data) == int:
        matrix[indices] = data
    else:
        for i in range(len(data)):
            insert(matrix, data[i], indices + (i,))
    

def pad_vector_data(data, pad_token, pad_shape=None):
    
    shape = size(data)
    if pad_shape != None:
        shape = tuple(np.maximum(pad_shape, shape))

    # empty matrix
    matrix = np.full(shape, pad_token, np.int32)

    insert(matrix, data, ())

    return matrix

### Split dataset

In [97]:
def split(data, *sizes):
    sizes = list(sizes)
    
    for i in range(1, len(sizes)):
        sizes[i] += sizes[i-1]
    
    slices = [slice(i,j) for i, j in zip([0]+sizes, sizes)]
    
    return [data[s] for s in slices]

### Shuffle dataset

In [99]:
def shuffle(*data):
    order = np.arange(len(data[0]))         # default order of elements
    np.random.shuffle(order)                # randomise order
    return [d[order] for d in data]         # new array with items in the randimised order

### Perform preprocessing

In [133]:
# extract LookupDOSFilePath for speccial processing

vectorized_data = {f: vectorize_data(data[f])    for f in data}
padded_data =     {f: pad_vector_data(vectorized_data[f], char_to_int['<Padding>'])    for f in vectorized_data}
onehot_data =     {f: keras.utils.to_categorical(padded_data[f], char_count)    for f in padded_data}
{f:onehot_data[f].shape for f in onehot_data}

{'Unique Record ID': (23903, 6, 53),
 'FileName': (23903, 87, 53),
 'Original_FileName': (23903, 71, 53),
 'SurveyNum': (23903, 5, 53),
 'SurveyName': (23903, 39, 53),
 'LineName': (23903, 23, 53),
 'SurveyType': (23903, 6, 53),
 'PrimaryDataType': (23903, 14, 53),
 'SecondaryDataType': (23903, 36, 53),
 'TertiaryDataType': (23903, 17, 53),
 'Quaternary': (23903, 8, 53),
 'File_Range': (23903, 13, 53),
 'First_SP_CDP': (23903, 8, 53),
 'Last_SP_CDP': (23903, 7, 53),
 'CompletionYear': (23903, 4, 53),
 'TenureType': (23903, 3, 53),
 'Operator Name': (23903, 47, 53),
 'GSQBarcode': (23903, 17, 53),
 'EnergySource': (23903, 29, 53),
 'LookupDOSFilePath': (23903, 181, 53),
 'Source Of Data': (23903, 8, 53),
 'LookupDOSFilePath_Words': (23903, 6, 87, 53),
 'FileName_Words': (23903, 13, 23, 53)}

### Extract relevant data

In [135]:
x_cut = slice(None, None)
y_cut = slice(None, None)

train_size = 5000
test_size = 1000
preview_size = 15

x_onehot = onehot_data['FileName']
y_onehot = onehot_data['LineName']

# x_count_words, x_count_chars, x_count_ones, *_ = (x_onehot.shape[1], x_onehot.shape[2], x_onehot.shape[3])
# y_count_words, y_count_chars, y_count_ones, *_ = (y_onehot.shape[1], y_onehot.shape[2], y_onehot.shape[3])
x_count_words, x_count_chars, x_count_ones, *_ = (None, x_onehot.shape[1], x_onehot.shape[2])
y_count_words, y_count_chars, y_count_ones, *_ = (None, y_onehot.shape[1], y_onehot.shape[2])

#x_onehot, y_onehot = shuffle(x_onehot, y_onehot)

x_train, x_test, x_preview = split(x_onehot, train_size, test_size, preview_size)
y_train, y_test, y_preview = split(y_onehot, train_size, test_size, preview_size)

x_train, y_train = shuffle(x_train, y_train)

{
'x_count_words': x_count_words, 'x_count_chars': x_count_chars, 'x_count_ones': x_count_ones, 
'y_count_words': y_count_words, 'y_count_chars': y_count_chars, 'y_count_ones': y_count_ones, 
}

{'x_count_words': None,
 'x_count_chars': 87,
 'x_count_ones': 53,
 'y_count_words': None,
 'y_count_chars': 23,
 'y_count_ones': 53}

## Test and show samlpe output

In [118]:
def test(model, x_test, y_test, x_preview, y_preview):
    p_one_hot = model.predict(x_preview)
    p_vector = np.argmax(p_one_hot, 2)
    p_strings = decode_data(p_vector)

    y_vector = np.argmax(y_preview, 2)
    y_strings = decode_data(y_vector)

    x_vector = np.argmax(x_preview, 2)
    x_strings = decode_data(x_vector)

    x_strings = [s.replace('<Padding>', ' ').strip() for s in x_strings]
    y_strings = [s.replace('<Padding>', ' ').strip() for s in y_strings]
    p_strings = [s.replace('<Padding>', ' ').strip() for s in p_strings]
    x_w, y_w, p_w = max([len(s) for s in x_strings]), max([len(s) for s in y_strings]), max([len(s) for s in p_strings])
    y_p_strings = ['  '.join([x.ljust(x_w), y.ljust(y_w), p.ljust(p_w), str(y==p)]) for x, y, p in zip(x_strings, y_strings, p_strings)]

    print(*y_p_strings, sep='\n', end='\n\n')

    # accuracy on entire training set
    accuracies = model.evaluate(x_test, y_test)
    print(*list(zip([loss]+metrics, accuracies)), sep='\n', end='\n\n') # evaluate and list loss and each metric
    
    return accuracies[0], accuracies[-1]

## Training Parameters

In [138]:
embedding_size = 15
character_embedding_size = 10
architecture = ''

metrics = ['mean_absolute_error', 'categorical_accuracy', 'binary_accuracy', exact_match_accuracy]
loss = 'mean_squared_logarithmic_error' # poisson mean_squared_logarithmic_error categorical_crossentropy

embed_loss='categorical_crossentropy'
embed_metrics=['accuracy', 'mean_absolute_error', 'categorical_accuracy', 'binary_accuracy']

models = {}

## Character Embedding

### Auto Encoder Character Data

In [136]:
# create offset input and output sequences to training a preditive embedding model.

# create columns of padding tokens
padding_train = np.array([[char_to_int['<Padding>']]] * train_size)
padding_train = keras.utils.to_categorical(padding_train, char_count)
padding_train = padding_train.reshape(train_size, 1, char_count)

padding_test = np.array([[char_to_int['<Padding>']]] * test_size)
padding_test = keras.utils.to_categorical(padding_test, char_count)
padding_test = padding_test.reshape(test_size, 1, char_count)

# 'abcd' -> ('_abcd', 'abcd_')
embed_train_x = np.concatenate((padding_train, x_train), axis=1)
embed_train_y = np.concatenate((x_train, padding_train), axis=1)
embed_test_x = np.concatenate((padding_test, x_test), axis=1)
embed_test_y = np.concatenate((x_test, padding_test), axis=1)

embed_char_count = embed_train_x.shape[1]

### Auto Encoder: Input, Hidden, Output

In [140]:
architecture = 'Character-Embedding'

model_E_D_NN = keras.Sequential()
model_E_D_NN.add(keras.layers.Dense(character_embedding_size, name='lh', input_shape=(embed_char_count, char_count,)))
model_E_D_NN.add(keras.layers.Dense(char_count, activation='sigmoid', name='lo'))
#model_E_D_NN.add(keras.layers.Dropout(0.001))
model_E_D_NN.compile(optimizer='adam', loss=embed_loss, metrics=embed_metrics)
models[architecture] = model_E_D_NN
print(model_E_D_NN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lh (Dense)                   (None, 88, 10)            540       
_________________________________________________________________
lo (Dense)                   (None, 88, 53)            583       
Total params: 1,123
Trainable params: 1,123
Non-trainable params: 0
_________________________________________________________________
None


### Train Encoder and Decoder

In [142]:
epochs = 5
batch_size = 64
model = models['Character-Embedding']

model.fit(embed_train_x, embed_train_x, batch_size=batch_size, epochs=epochs)

accuracy = model.evaluate(embed_test_x, embed_test_x)
metric_names = [embed_loss] + embed_metrics
dict(zip(metric_names, accuracy))

Epoch 1/5
5000/5000 [==============================] - 1s 192us/step - loss: 1.5529 - acc: 0.3988 - mean_absolute_error: 0.0592 - categorical_accuracy: 0.3988 - binary_accuracy: 0.9787
Epoch 2/5
5000/5000 [==============================] - 1s 195us/step - loss: 1.2991 - acc: 0.3988 - mean_absolute_error: 0.0484 - categorical_accuracy: 0.3988 - binary_accuracy: 0.9816
Epoch 3/5
5000/5000 [==============================] - 1s 188us/step - loss: 1.1087 - acc: 0.3988 - mean_absolute_error: 0.0403 - categorical_accuracy: 0.3988 - binary_accuracy: 0.9841
Epoch 4/5
5000/5000 [==============================] - 1s 191us/step - loss: 0.9554 - acc: 0.4210 - mean_absolute_error: 0.0338 - categorical_accuracy: 0.4210 - binary_accuracy: 0.9851
Epoch 5/5
1000/1000 [==============================] - 0s 98us/step


{'categorical_crossentropy': 0.7513767862319947,
 'accuracy': 0.90248863363266,
 'mean_absolute_error': 0.0266893962174654,
 'categorical_accuracy': 0.90248863363266,
 'binary_accuracy': 0.9934578099250794}

In [160]:
# w1 = model.layers[0].get_weights()
# w2 = model.layers[1].get_weights()

# w = [np.copy(w1[0]), np.zeros(w1[1].shape)]
# wi = [np.linalg.pinv(w1[0]), np.zeros(w2[1].shape)]

# m = keras.Sequential()
# m.add(keras.layers.Dense(character_embedding_size, activation='linear', name='lh', input_shape=(embed_char_count, voc_size,)))
# m.add(keras.layers.Dense(voc_size, activation='sigmoid', name='lo'))
# m.compile(optimizer='adam', loss=embed_loss, metrics=embed_metrics)

# m.layers[0].set_weights(w)
# m.layers[1].set_weights(wi)

# encode_weights, decode_weights = w, wi

# accuracy = model.evaluate(embed_test_x, embed_test_x)
# metric_names = [embed_loss] + embed_metrics
# dict(zip(metric_names, accuracy))
char_encode_weights, char_decode_weights = models['Character-Embedding'].layers[0].get_weights(), models['Character-Embedding'].layers[1].get_weights()

## Folder Embedding

### Auto Encoder Folder Data

In [ ]:
# split text by '\' to seperate into 'words'
# resulting data shape (dataset_size, max_words_per_sample, max_chars_per_word, categorical_onehot)


### Auto Encoder Model: Input, Hidden, Output

### Train Encoder and Decoder

## Models

### P-NN: Input, Embedding, Output

In [1]:
architecture = 'P-NN'

model_P_NN = keras.Sequential()
model_P_NN.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
model_P_NN.add(keras.layers.Flatten())                                                                       # flatten to 1D per sample
model_P_NN.add(keras.layers.Dense(y_shape_char*y_shape_ones, activation='exponential', name='lo'))           # dense layer
model_P_NN.add(keras.layers.Dropout(0.001))                                                                  # dropout to prevent overfitting
model_P_NN.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_P_NN.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_P_NN
print(model_P_NN.summary())

NameError: name 'keras' is not defined

### FF-NN: Input, Embedding, Hidden, Output

In [37]:
architecture = 'FF-NN'
hidden_size = (y_shape_ones*embedding_size + y_shape_char*y_shape_ones) // 2

model_FF_NN = keras.Sequential()
model_FF_NN.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
model_FF_NN.add(keras.layers.Flatten())                                                                       # flatten to 1D per sample
model_FF_NN.add(keras.layers.Dense(hidden_size, activation='exponential', name='lh'))                         # dense layer
model_FF_NN.add(keras.layers.Dropout(0.2))                                                                    # dropout to prevent overfitting
model_FF_NN.add(keras.layers.Dense(y_shape_char*y_shape_ones, activation='exponential', name='lo'))           # dense layer
model_FF_NN.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_FF_NN.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_FF_NN
print(model_FF_NN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
le (Embedding)               (None, 23, 20)            1580      
_________________________________________________________________
flatten_5 (Flatten)          (None, 460)               0         
_________________________________________________________________
lh (Dense)                   (None, 1264)              582704    
_________________________________________________________________
dropout_6 (Dropout)          (None, 1264)              0         
_________________________________________________________________
lo (Dense)                   (None, 948)               1199220   
_________________________________________________________________
reshape_7 (Reshape)          (None, 12, 79)            0         
Total params: 1,783,504
Trainable params: 1,783,504
Non-trainable params: 0
_________________________________________________________________


### LSTM-RNN1: Input, Embedding, (LSTM), Output

In [83]:
architecture = 'LSTM-RNN1'
lstm_hidden_size = embedding_size * 15

model_LSTM_RNN1 = keras.Sequential()
model_LSTM_RNN1.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
#model_LSTM_RNN1.add(keras.layers.Dropout(0.2))                                                                    # dropout to prevent overfitting
model_LSTM_RNN1.add(keras.activation.exponential())
model_LSTM_RNN1.add(keras.layers.LSTM(y_shape_char * y_shape_ones, activation='exponential', implementation=2, unroll=True))                # lstm recurrent cell
model_LSTM_RNN1.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_LSTM_RNN1.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_LSTM_RNN1
print(model_LSTM_RNN1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
le (Embedding)               (None, 23, 20)            1580      
_________________________________________________________________
lstm_7 (LSTM)                (None, 948)               3674448   
_________________________________________________________________
reshape_15 (Reshape)         (None, 12, 79)            0         
Total params: 3,676,028
Trainable params: 3,676,028
Non-trainable params: 0
_________________________________________________________________
None


### LSTM-RNN2: Input, Embedding, (LSTM), Output

In [18]:
architecture = 'LSTM-RNN2'
lstm_hidden_size = embedding_size * 15

model_LSTM_RNN2 = keras.Sequential()
model_LSTM_RNN2.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
model_LSTM_RNN2.add(keras.layers.Dropout(0.2))                                                                    # dropout to prevent overfitting
model_LSTM_RNN2.add(keras.layers.LSTM(lstm_hidden_size, return_sequences=True, return_state=True))                # lstm recurrent cell
model_LSTM_RNN2.add(keras.layers.Dropout(0.2))                                                                    # dropout to prevent overfitting
model_LSTM_RNN2.add(keras.layers.Dense(y_shape_char * y_shape_ones))                                              # dense combine time series into single output
model_LSTM_RNN2.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_LSTM_RNN2.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_LSTM_RNN2
print(model_LSTM_RNN2.summary())

TypeError: All layers in a Sequential model should have a single output tensor. For multi-output layers, use the functional API.

### GRU-RNN1: Input, Embedding, (GRU), Output

In [152]:
architecture = 'GRU-RNN1'
lstm_hidden_size = voc_size * 15

model_GRU_RNN1 = keras.Sequential()
model_GRU_RNN1.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
#model_GRU_RNN1.add(keras.layers.Dropout(0.2))                                                                    # dropout to prevent overfitting
model_GRU_RNN1.add(keras.layers.GRU(y_shape_char * y_shape_ones, activation='relu', implementation=2, unroll=True))                # lstm recurrent cell
model_GRU_RNN1.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_GRU_RNN1.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_GRU_RNN1
print(model_GRU_RNN1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
le (Embedding)               (None, 23, 20)            1580      
_________________________________________________________________
gru_8 (GRU)                  (None, 948)               2755836   
_________________________________________________________________
reshape_24 (Reshape)         (None, 12, 79)            0         
Total params: 2,757,416
Trainable params: 2,757,416
Non-trainable params: 0
_________________________________________________________________
None


### GRU-RNN2: Imput Embedding, (GRU), Decoder, Output

In [161]:
architecture = 'GRU-RNN2'
lstm_hidden_size = y_shape_char * character_embedding_size + 40

model_GRU_RNN2 = keras.Sequential()
model_GRU_RNN2.add(keras.layers.Dense(character_embedding_size, name='char_encode', trainable=False, input_shape=(x_shape_char, x_shape_ones)))                 # embed characters into dense embedded space
#model_GRU_RNN2.add(keras.layers.Dropout(0.2))                                                                            # dropout to prevent overfitting
model_GRU_RNN2.add(keras.layers.GRU(lstm_hidden_size, activation='sigmoid', implementation=2, unroll=True, name='gru1'))     # gru recurrent cell
#model_GRU_RNN2.add(keras.layers.Dropout(0.2))                                                                            # dropout to prevent overfitting
model_GRU_RNN2.add(keras.layers.Dense(y_shape_char * character_embedding_size, activation='sigmoid', name='decode'))  # dense layer, decode/de-embed
model_GRU_RNN2.add(keras.layers.Reshape((y_shape_char, character_embedding_size)))                                        # un flatten
model_GRU_RNN2.add(keras.layers.Dense(y_shape_ones, activation='sigmoid', trainable=False, name='char_decode'))                             # dense layer, decode/de-embed
model_GRU_RNN2.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_GRU_RNN2
print(model_GRU_RNN2.summary())

# set pretrained embedding weights
# model_GRU_RNN2.layers[0].set_weights( models['Character-Embedding'].layers[0].get_weights())
# model_GRU_RNN2.layers[-1].set_weights(models['Character-Embedding'].layers[-1].get_weights())
model_GRU_RNN2.layers[0].set_weights(char_encode_weights)
model_GRU_RNN2.layers[-1].set_weights(char_decode_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_encode (Dense)          (None, 87, 10)            800       
_________________________________________________________________
gru1 (GRU)                   (None, 160)               82080     
_________________________________________________________________
decode (Dense)               (None, 120)               19320     
_________________________________________________________________
reshape_15 (Reshape)         (None, 12, 10)            0         
_________________________________________________________________
char_decode (Dense)          (None, 12, 79)            869       
Total params: 103,069
Trainable params: 101,400
Non-trainable params: 1,669
_________________________________________________________________
None


## Save/Restore weights

In [184]:
#DE = models['E-D-NN'].get_weights()
#model_GRU_1 = model
#model_GRU_2 = model
#model_GRU_3 = model

In [133]:
#model.set_weights(GRU)
#model = model_GRU_3
#models['E-D-NN'].set_weights(DE)

## Run and Evaluate

In [162]:
epochs = 1
batch_size = 128
model = models['GRU-RNN2']

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs)
l, a = test(model, test_x, test_y, showcase_x, showcase_y)
training_log(x_name, y_name, architecture, batch_size, epochs, l, a)
print(*training_history[::-1], sep='\n')

Epoch 1/1
4000/4000 [==============================] - 233s 58ms/step - loss: 0.0820 - mean_absolute_error: 0.2712 - categorical_accuracy: 0.0070 - binary_accuracy: 0.7966 - exact_match_accuracy: 0.0000e+00
XYLEM_XM86-7_STACK_SDU10912TA_130263.sgy             XM86-7   LLLLLLLLLLLL  False
METEOR_79-D20A_FINAL_MIGRATED_SDU10912TA_129215.sgy  79-D20A  LLLLLLLLLLLL  False
YOOTHAPINNA_83-E307_STACK_SDU10912TA_129872.sgy      83-E307  LLLLLLLLLLLL  False
PIEBALD_MARMADUA_80-H14_STACK_SDU10912TA_129292.sgy  80-H14   LLLLLLLLLLLL  False
XYLEM_XM86-4_STACK_SDU10912TA_130260.sgy             XM86-4   LLLLLLLLLLLL  False

1000/1000 [==============================] - 37s 37ms/step
('mean_squared_logarithmic_error', 0.016132571235299112)
('mean_absolute_error', 0.09493823343515397)
('categorical_accuracy', 0.012416666604578495)
('binary_accuracy', 0.9749974575042725)
(<function exact_match_accuracy at 0x0000026AC9ECCC80>, 0.0)

{'x': 'FileName', 'y': 'LineName', 'architecture': 'GRU-RNN2', 'batch si

In [109]:
print(*training_history[::-1], sep='\n')

{'x': 'FileName', 'y': 'LineName', 'architecture': 'GRU-RNN2', 'batch size': 128, 'epochs': 10, 'loss': 0.004683957852423191, 'accuracy': 0.0}
{'x': 'FileName', 'y': 'LineName', 'architecture': 'GRU-RNN2', 'batch size': 128, 'epochs': 10, 'loss': 0.004685713063925505, 'accuracy': 0.0}
{'x': 'FileName', 'y': 'LineName', 'architecture': 'GRU-RNN2', 'batch size': 128, 'epochs': 10, 'loss': 0.0038579754158854485, 'accuracy': 0.0}
{'x': 'FileName', 'y': 'LineName', 'architecture': 'GRU-RNN2', 'batch size': 128, 'epochs': 10, 'loss': 0.005442343842238188, 'accuracy': 0.0}
{'x': 'FileName', 'y': 'LineName', 'architecture': 'GRU-RNN2', 'batch size': 128, 'epochs': 1, 'loss': 0.005693509742617607, 'accuracy': 0.0}
{'x': 'FileName', 'y': 'LineName', 'architecture': 'GRU-RNN2', 'batch size': 128, 'epochs': 1, 'loss': 0.009779514268040657, 'accuracy': 0.0}
